
# 01. SETUP 📦


In [ ]:
!pip install pandas geopandas folium plotly

import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px
from google.colab import files

# 02. UPLOAD CSV FILE 📂

In [ ]:
# Upload CSV file
uploaded_csv = files.upload()


Saving Mumbai Roads.csv to Mumbai Roads.csv


# 03. LOAD DATA 🧹

In [ ]:
try:
    # Try to load as GeoDataFrame (if shapely geometries exist)
    gdf = gpd.read_file("Mumbai Roads.csv")
except Exception:
    # Otherwise, fallback to pandas DataFrame
    df = pd.read_csv("Mumbai Roads.csv")
    # Try to find coordinate columns (commonly lat/lon or x/y)
    possible_lat = [c for c in df.columns if 'lat' in c.lower()]
    possible_lon = [c for c in df.columns if 'lon' in c.lower()]
    if possible_lat and possible_lon:
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[possible_lon[0]], df[possible_lat[0]]))
    else:
        raise ValueError("⚠️ No coordinate columns (lat/lon) found. Please include them in your CSV.")

print("✅ Data loaded successfully!")
print("Columns:", gdf.columns.tolist())
print("Unique fclass values:", gdf['fclass'].unique()[:23])

✅ Data loaded successfully!
Columns: ['fid', 'osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed', 'layer', 'bridge', 'tunnel']
Unique fclass values: ['primary' 'unclassified' 'tertiary' 'residential' 'motorway' 'secondary'
 'trunk' 'motorway_link' 'service' 'path' 'living_street' 'footway'
 'primary_link' 'secondary_link' 'pedestrian' 'track' 'trunk_link' 'steps'
 'tertiary_link' 'bridleway' 'track_grade2' 'cycleway' 'track_grade3']


# 04. DATA FILTERATION 🧹

In [ ]:
selected_fclasses = [
    'primary', 'unclassified', 'tertiary', 'residential', 'motorway',
    'secondary', 'trunk', 'service', 'path', 'living_street',
    'footway', 'pedestrian', 'track', 'steps', 'bridleway', 'cycleway'
]  # <-- change this list as needed
filtered_gdf = gdf[gdf['fclass'].isin(selected_fclasses)]

print(f"Filtered GeoDataFrame with fclass in {selected_fclasses}:")
print(filtered_gdf.head())

Filtered GeoDataFrame with fclass in ['primary', 'unclassified', 'tertiary', 'residential', 'motorway', 'secondary', 'trunk', 'service', 'path', 'living_street', 'footway', 'pedestrian', 'track', 'steps', 'bridleway', 'cycleway']:
  fid   osm_id  code        fclass                                 name ref  \
0   1  8458446  5113       primary                   Kalyan - Shil Road       
1   2  8458456  5113       primary                   Kalyan - Shil Road       
2   3  8461852  5121  unclassified  Shree Nateshwar Mahadev Mandir Marg       
3   4  8462541  5115      tertiary                   MIDC Pipeline Road       
4   5  8462602  5113       primary                   Kalyan - Shil Road       

  oneway maxspeed layer bridge tunnel  
0      F        0     0      F      F  
1      F        0     0      F      F  
2      B        0     0      F      F  
3      B        0     0      F      F  
4      F        0     0      F      F  



# 05. ANALYTICAL VISUALIZATION 📊


In [ ]:
import plotly.express as px

# ======================
fclass_counts = filtered_gdf['fclass'].value_counts().reset_index()
fclass_counts.columns = ['fclass', 'count']

# --- Bar Chart with Viridis Gradient ---
fig_bar = px.bar(
    fclass_counts, x='fclass', y='count',
    color='count',  # color based on count
    color_continuous_scale='Viridis',  # each bar a different shade
    text='count',
    title="Distribution of Selected Road Types in Mumbai"
)

fig_bar.update_traces(
    textposition='outside',
    marker_line_color="white",  # subtle glow
    marker_line_width=1.2
)

fig_bar.update_layout(
    xaxis_title="Road Type",
    yaxis_title="Number of Roads",
    xaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.05)", tickangle=0),  # horizontal labels
    yaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.05)"),
    plot_bgcolor="rgb(10,10,10)",
    paper_bgcolor="rgb(15,15,15)",
    font_color="white",
    bargap=0.2
)

fig_bar.show()

# --- Treemap ---
fig_treemap = px.treemap(
    fclass_counts, path=['fclass'], values='count',
    color='count', color_continuous_scale='Viridis',
    title="Treemap of Selected Road Classifications"
)
fig_treemap.show()
